In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Assignment").getOrCreate()
type(spark)

pyspark.sql.session.SparkSession

In [18]:
#1.Creating Resilient Distributed Dataset in 3 different ways.

#Method 1:Parallelize

text = "Your time is limited, so don't waste it living someone else's life. Don't be trapped by dogma – which is living with the results of other people's thinking.".split(" ")
words = spark.sparkContext.parallelize(text,2)
typ = type(words)
word = words.collect()
count = words.count()

print("type of words :",typ)
print("Number of words :",count)
print(word)

type of words : <class 'pyspark.rdd.RDD'>
Number of words : 28
['Your', 'time', 'is', 'limited,', 'so', "don't", 'waste', 'it', 'living', 'someone', "else's", 'life.', "Don't", 'be', 'trapped', 'by', 'dogma', '–', 'which', 'is', 'living', 'with', 'the', 'results', 'of', 'other', "people's", 'thinking.']


In [25]:
#Method 2: Using Data source

data = spark.sparkContext.textFile("textfile.txt")
Data = data.collect()
t2 = type(Data)
print("type of Data",t2)
print(Data)

type of Data <class 'list'>
['The greatest glory in living lies not in never falling, but in rising every time we fall.', 'The way to get started is to quit talking and begin doing.', "Your time is limited, so don't waste it living someone else's life. Don't be trapped by dogma – which is living with the results of other people's thinking.", 'If life were predictable it would cease to be life, and be without flavor.', "If you look at what you have in life, you'll always have more. If you look at what you don't have in life, you'll never have enough.", "If you set your goals ridiculously high and it's a failure, you will fail above everyone else's success.    ", "Life is what happens when you're busy making other plans."]


In [27]:
#Method 3: Using Pipelined RDD
info = words.filter(lambda word:word.startswith('s'))
inf = info.collect()
print(type(info))
print(inf)

<class 'pyspark.rdd.PipelinedRDD'>
['so', 'someone']


In [28]:
#2.Read a text file and count the number of words in the file using RDD operations.

Total_words=data.flatMap(lambda s:s.split(" "))
Total_words.count()


130

In [29]:
#3.Write a program to find the word frequency in a given file.
datainfo=spark.sparkContext.textFile("textfile.txt")
datainfo=datainfo.flatMap(lambda x:x.split())
content=datainfo.map(lambda c:(c,1))
content.reduceByKey(lambda a,b:a+b).collect()

[('The', 2),
 ('greatest', 1),
 ('glory', 1),
 ('in', 5),
 ('living', 3),
 ('never', 2),
 ('falling,', 1),
 ('but', 1),
 ('rising', 1),
 ('we', 1),
 ('fall.', 1),
 ('way', 1),
 ('started', 1),
 ('is', 4),
 ('quit', 1),
 ('talking', 1),
 ('doing.', 1),
 ('life.', 1),
 ("Don't", 1),
 ('dogma', 1),
 ('results', 1),
 ('of', 1),
 ('other', 2),
 ("people's", 1),
 ('thinking.', 1),
 ('would', 1),
 ('cease', 1),
 ('life,', 3),
 ('look', 2),
 ('at', 2),
 ('have', 4),
 ("you'll", 2),
 ('always', 1),
 ('set', 1),
 ('ridiculously', 1),
 ('high', 1),
 ('above', 1),
 ('success.', 1),
 ('Life', 1),
 ('when', 1),
 ("you're", 1),
 ('making', 1),
 ('lies', 1),
 ('not', 1),
 ('every', 1),
 ('time', 2),
 ('to', 3),
 ('get', 1),
 ('and', 3),
 ('begin', 1),
 ('Your', 1),
 ('limited,', 1),
 ('so', 1),
 ("don't", 2),
 ('waste', 1),
 ('it', 2),
 ('someone', 1),
 ("else's", 2),
 ('be', 3),
 ('trapped', 1),
 ('by', 1),
 ('–', 1),
 ('which', 1),
 ('with', 1),
 ('the', 1),
 ('If', 4),
 ('life', 1),
 ('were', 1),
 

In [30]:
#4. Write a program to convert all words in a file to uppercase.

data.map(lambda c:c.upper()).collect()

['THE GREATEST GLORY IN LIVING LIES NOT IN NEVER FALLING, BUT IN RISING EVERY TIME WE FALL.',
 'THE WAY TO GET STARTED IS TO QUIT TALKING AND BEGIN DOING.',
 "YOUR TIME IS LIMITED, SO DON'T WASTE IT LIVING SOMEONE ELSE'S LIFE. DON'T BE TRAPPED BY DOGMA – WHICH IS LIVING WITH THE RESULTS OF OTHER PEOPLE'S THINKING.",
 'IF LIFE WERE PREDICTABLE IT WOULD CEASE TO BE LIFE, AND BE WITHOUT FLAVOR.',
 "IF YOU LOOK AT WHAT YOU HAVE IN LIFE, YOU'LL ALWAYS HAVE MORE. IF YOU LOOK AT WHAT YOU DON'T HAVE IN LIFE, YOU'LL NEVER HAVE ENOUGH.",
 "IF YOU SET YOUR GOALS RIDICULOUSLY HIGH AND IT'S A FAILURE, YOU WILL FAIL ABOVE EVERYONE ELSE'S SUCCESS.    ",
 "LIFE IS WHAT HAPPENS WHEN YOU'RE BUSY MAKING OTHER PLANS."]

In [31]:
#5. Write a program to convert all words in a file to lowercase.

data.map(lambda c:c.lower()).collect()

['the greatest glory in living lies not in never falling, but in rising every time we fall.',
 'the way to get started is to quit talking and begin doing.',
 "your time is limited, so don't waste it living someone else's life. don't be trapped by dogma – which is living with the results of other people's thinking.",
 'if life were predictable it would cease to be life, and be without flavor.',
 "if you look at what you have in life, you'll always have more. if you look at what you don't have in life, you'll never have enough.",
 "if you set your goals ridiculously high and it's a failure, you will fail above everyone else's success.    ",
 "life is what happens when you're busy making other plans."]

In [32]:
#6. Write a program to capitalize first letter of each words in file (use string capitalize() method)

Uppercase = data.flatMap(lambda a:a.split(" ")).map(lambda c:c.capitalize()).collect()
" ".join(Uppercase)


"The Greatest Glory In Living Lies Not In Never Falling, But In Rising Every Time We Fall. The Way To Get Started Is To Quit Talking And Begin Doing. Your Time Is Limited, So Don't Waste It Living Someone Else's Life. Don't Be Trapped By Dogma – Which Is Living With The Results Of Other People's Thinking. If Life Were Predictable It Would Cease To Be Life, And Be Without Flavor. If You Look At What You Have In Life, You'll Always Have More. If You Look At What You Don't Have In Life, You'll Never Have Enough. If You Set Your Goals Ridiculously High And It's A Failure, You Will Fail Above Everyone Else's Success.     Life Is What Happens When You're Busy Making Other Plans."

In [33]:
#7.Find the longest length of word from given set of words.

longestlength=data.flatMap(lambda x:x.split(" "))
longestlength.map(lambda nu:(len(nu),nu)).max()[1]

'ridiculously'

In [34]:
#8. Map the Registration numbers to corresponding branch.
#6000 series BDA, 9000 series HDA, 1000 series ML, 2000 series VLSI, 3000 series ES, 4000 series MSc, 5000 series CC.
#Given registration number, generate a key-value pair of Registration Number and Corresponding Branch.

USN = [6001,2001,2002,5001,6002,9001,3001,3002,4001,4002,5002,5003,1001,1002]
context=spark.sparkContext.parallelize(USN,2)
classify=context.map(lambda reg:('VLSI',reg) if reg>2000 and reg<3000 
        else ('MS',reg) if reg>1000 and reg<2000
        else ('ES',reg) if reg>3000 and reg<4000
        else ('MSc',reg) if reg>4000 and reg<5000
        else ('CC',reg) if reg>5000 and reg<6000
        else ('BDA',reg) if reg>6000 and reg<7000
        else ('HDA',reg))
classified_usn = classify.collect()
print(classified_usn)

[('BDA', 6001), ('VLSI', 2001), ('VLSI', 2002), ('CC', 5001), ('BDA', 6002), ('HDA', 9001), ('ES', 3001), ('ES', 3002), ('MSc', 4001), ('MSc', 4002), ('CC', 5002), ('CC', 5003), ('MS', 1001), ('MS', 1002)]


In [50]:
# 9.
#Text file contain numbers. Numbers are separated by one white space. There is no order to store the numbers.
#One line may contain one or more numbers. Find the maximum, minimum, sum and mean of numbers.
num=spark.sparkContext.textFile("Numbers.txt")
num_rdd=num.flatMap(lambda z:z.split(" ")).map(lambda c:int(c))
print("Maximum number",num_rdd.max())
print("Minimum number",num_rdd.min())
print("Sum",num_rdd.sum())
print("Mean",num_rdd.mean())

Maximum number 90
Minimum number 12
Sum 1364
Mean 48.714285714285715


In [51]:
#10. A text file (citizen.txt) contains data about citizens of country. Fields (information in file) are Name, dob, Phone,
# email and state name. Another file contains mapping of state names to state code like Karnataka is codes as KA, TamilNadu as TN, Kerala KL etc.
# Compress the citizen.txt file by changing full state name to state code.

Citizen_details = spark.sparkContext.textFile("citizen.txt")
State_code = spark.sparkContext.textFile("State_codes.txt")

details_rdd = Citizen_details.map(lambda x:x.split(",")).collect()
code_rdd= State_code.map(lambda y:y.split(",")).collect()

for i in range(len(details_rdd)):
    for j in range(len(code_rdd)):  
        if details_rdd[i][4]==code_rdd[j][0]:
            details_rdd[i][4]=code_rdd[j][1]
details_rdd     




[['Thejas', '25-5-1997', '201046013', 'thejas@gmail.com', 'AP'],
 ['Rakhesh', '22-03-1997', '201051023', 'rakhi@gmail.com', 'MH'],
 ['Shanthkumar', '14-03-1999', '201046049', 'shanthu@gmail.com', 'GJ'],
 ['kiran', '21-10-1997', '2010156001', 'kiran@gmail.com', 'WestBengal'],
 ['Anupam', '05-09-1993', '201056002', 'anupam@gmail.com', 'RJ'],
 ['Subrahmanyan', '04-12-1997', '201046007', 'sub@gmail.com', 'KA'],
 ['Abhishek', '15-10-1997', '201051007', 'abhi@gmail.com', 'DL'],
 ['Chandan', '13-02-1997', '201046003', 'chandan@gmail.com', 'PB']]

In [52]:
state = spark.sparkContext.textFile('State_codes.txt')
stateKey = state.map(lambda word: (word.split(',')[0], word.split(',')[1]))

Statecode_dict = {}
for val in stateKey.collect():
    Statecode_dict[val[0]] = val[1]
    
Statecode_dict

{'Karnataka': 'KA',
 'Delhi': 'DL',
 'Gujarat': 'GJ',
 'Maharashtra': 'MH',
 'Rajasthan': 'RJ',
 'Punjab': 'PB',
 'AndhraPradesh': 'AP',
 'westBengal': 'WB'}

In [53]:
mapData = spark.sparkContext.broadcast(Statecode_dict)

city = spark.sparkContext.textFile('citizen.txt')
print(city.collect())
def compress(state,codes):
    splitData = state.split(',')  
    print(splitData)
    splitData[4] = codes.value.get(splitData[4])
    newData = ' '
    newData = newData.join(splitData)
    
    return newData
    
Citizen = city.map(lambda data: compress(data,mapData))



['Thejas,25-5-1997,201046013,thejas@gmail.com,AndhraPradesh', 'Rakhesh,22-03-1997,201051023,rakhi@gmail.com,Maharashtra', 'Shanthkumar,14-03-1999,201046049,shanthu@gmail.com,Gujarat', 'kiran,21-10-1997,2010156001,kiran@gmail.com,WestBengal', 'Anupam,05-09-1993,201056002,anupam@gmail.com,Rajasthan', 'Subrahmanyan,04-12-1997,201046007,sub@gmail.com,Karnataka', 'Abhishek,15-10-1997,201051007,abhi@gmail.com,Delhi', 'Chandan,13-02-1997,201046003,chandan@gmail.com,Punjab']
